In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
from tqdm import tqdm_notebook
import concurrent.futures
from multiprocessing import Pool

# 【方案一】Wiki系列API

## wikipedia
- wikipedia | https://pypi.org/project/wikipedia/ 

In [7]:
import wikipedia
wikipedia.set_lang("en")
wikipedia.summary("Bottle")
wikipedia.summary("Bottle",sentences=1)
wikipedia.search("Bottle") # 检索Barack

'A bottle is a narrow-necked container made of an impermeable material (clay, glass, plastic, aluminium, etc.) in various shapes and sizes  to store and transport liquids (water, milk, beer, wine, ink, cooking oil, medicine, soft drinks, shampoo, and chemicals, etc.) and whose mouth at the bottling line can be sealed with an internal stopper, an external bottle cap, a closure, or a conductive "inner seal" using induction sealing.\nSome of the earliest bottles appeared in China, Phoenicia, Crete, and Rome.'

'A bottle is a narrow-necked container made of an impermeable material (clay, glass, plastic, aluminium, etc.) in various shapes and sizes  to store and transport liquids (water, milk, beer, wine, ink, cooking oil, medicine, soft drinks, shampoo, and chemicals, etc.) and whose mouth at the bottling line can be sealed with an internal stopper, an external bottle cap, a closure, or a conductive "inner seal" using induction sealing.'

['Bottle',
 'Bottle It In',
 'Coca-Cola',
 'Klein bottle',
 'Polyethylene terephthalate',
 'Bottle Rocket',
 'The Bottle',
 'Wine bottle',
 'Bottle episode',
 'Calabash']

In [42]:
try:
    ny = wikipedia.page("China") # 直接获取页面
    ny.title
    ny.url
    len(ny.content)
    len(ny.links)
    ny.links[0]
except wikipedia.DisambiguationError as e:
    print("目标词条有歧义")
except wikipedia.WikipediaException as e:
    print("未找到目标词条")

'China'

'https://en.wikipedia.org/wiki/China'

98415

1963

'+86'

## wikipediaapi
- wikipedia-API | https://pypi.org/project/Wikipedia-API/

In [ ]:
import wikipediaapi
wiki_wiki = wikipediaapi.Wikipedia('en')
page_py = wiki_wiki.page('Python_(programming_language)')

page_py.exists() # 检查页面是否存在
page_py.title
page_py.summary # 获取summary
page.sections # 获取sections
page.sections[0].sections # 获取二级section
page_py.canonicalurl # 获得页面的url | 区别未知
page_py.fullurl # 获得页面的url | 区别未知
page_py.text # 内容
page_py_an = page_py.langlinks['an'] # 获取当前页面其他语言形式
cats = page.categories # 获取category

# 内容（使用html形式）
wiki_html=wikipediaapi.Wikipedia(language='en', extract_format=wikipediaapi.ExtractFormat.HTML)
wiki_html.page("India").text

# 获取页面中所有连接（指向其他wiki页面的链接）
links = page_py.links
for title in sorted(links.keys()):
    print(title,links[title])

### 获取category下所有的页面

In [ ]:
# 获取category在的所有页面
def print_categorymembers(categorymembers, level=0, max_level=1):
        for c in categorymembers.values():
            print("%s: %s (ns: %d)" % ("*" * (level + 1), c.title, c.ns))
            if c.ns == wikipediaapi.Namespace.CATEGORY and level < max_level:
                print_categorymembers(c.categorymembers, level=level + 1, max_level=max_level)

cat = wiki_wiki.page("Category:Physics")
print("Category members: Category:Physics")
print_categorymembers(cat.categorymembers)

### 流程

In [25]:
import wikipediaapi
wiki_wiki = wikipediaapi.Wikipedia('en')

In [109]:
class WikiSearch():
    @staticmethod
    def search(targetWord= "BJP"):
        page_py = wiki_wiki.page(targetWord)
        if page_py.exists():
            title = page_py.title
            url = page_py.canonicalurl
            cat = [i.title.split("Category:")[1] for i in page_py.categories.values()]
        else:
            title = ""
            url = ""
            cat = []
            print("page not exists")
        return (title,url,cat)

In [110]:
WikiSearch.search("China")

('China',
 'https://en.wikipedia.org/wiki/China',
 ['All Wikipedia articles written in American English',
  'All articles containing potentially dated statements',
  'All articles with vague or ambiguous time',
  'Articles containing Chinese-language text',
  'Articles containing Mongolian-language text',
  'Articles containing Persian-language text',
  'Articles containing Portuguese-language text',
  'Articles containing Sanskrit-language text',
  'Articles containing potentially dated statements from 2005',
  'Articles containing potentially dated statements from 2010',
  'Articles containing potentially dated statements from 2012',
  'Articles containing potentially dated statements from 2013',
  'Articles containing potentially dated statements from 2015',
  'Articles containing potentially dated statements from 2017',
  'Articles containing potentially dated statements from 2018',
  'Articles containing potentially dated statements from January 2016',
  'Articles containing simpl

In [98]:
page_cat = list(page_py.categories.values())[0]
type(page_cat)
type(page_py)
page_cat.title
page_py.ns
page_cat.ns
wikipediaapi.Namespace.CATEGORY
page_cat.links

wikipediaapi.WikipediaPage

wikipediaapi.WikipediaPage

'Category:All Wikipedia articles written in American English'

0

14

<Namespace.CATEGORY: 14>

{'Wikipedia:Categorization': Wikipedia:Categorization (id: ??, ns: 4),
 'Wikipedia:Maintenance': Wikipedia:Maintenance (id: ??, ns: 4),
 'Wikipedia:What is an article?': Wikipedia:What is an article? (id: ??, ns: 4),
 'Help:Category': Help:Category (id: ??, ns: 12),
 'Category:Hidden categories': Category:Hidden categories (id: ??, ns: 14)}

# 【方案二】纯爬虫处理